In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle

Using cache found in /mnt/d/torch_data/hub/huggingface_pytorch-transformers_main


In [2]:
character_flag_generators = [
            lambda c: int(c.isupper()),
            lambda c: int(c.lower() in ["ä", "ö", "ü", "ß"]),
            lambda c: int(c.isdigit()),
            lambda c: int(c in r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""")
        ]

In [6]:
dataset_generator = TigerDatasetGenerator(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", (0.1, 0.1), character_flag_generators=character_flag_generators, prop_of_tiger_to_use=0.4) # type:ignore

INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsed 20188 sentences.
INFO:model:Generating trees...
INFO:model:16885 (83.64%) trees generated.
INFO:model:Dataset split into 13508 training, 1688 dev, and 1689 test trees.


In [7]:
batch_size = 64

required_vars = (dataset_generator.get_training_dataloader(batch_size=batch_size, shuffle=True),
                 dataset_generator.get_dev_dataloader(batch_size=batch_size, shuffle=False),
                 dataset_generator.get_test_dataloader(batch_size=batch_size, shuffle=False),
                 dataset_generator.character_set,
                 dataset_generator.character_flag_generators,
                 dataset_generator.inverse_word_dict,
                 dataset_generator.inverse_sym_set,
                 dataset_generator.inverse_pos_set,
                 dataset_generator.inverse_morph_dicts
                )

In [5]:
pickle.dump(required_vars, open("required_vars.pkl", "wb"))

In [6]:
(train_dataloader, train_new_words), (dev_dataloader, dev_new_words), (test_dataloader, test_new_words), character_set, character_flag_generators, inverse_word_dict, inverse_sym_dict, inverse_pos_dict, inverse_morph_dicts = pickle.load(open("required_vars.pkl", "rb"))